## 정해진 변수로 간단히 모델 돌려보기
- Catboost, XGBoost, SVC, NB, 랜덤포레스트, LSTM

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import itertools
import numpy as np


from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout

In [19]:
# 변수들 불러오기
Vari_13 = pd.read_csv("data/13_variable_close.csv")
NBC_N = pd.read_csv("data/NLTK_NBC_Score_DowJ.csv")
NBC_TB = pd.read_csv("data/TextB_NBC_Score_DowJ.csv")
NBC_B = pd.read_csv("data/BERT_NBC_Score_DowJ.csv")
NYT_N = pd.read_csv("data/NLTK_NYT_Score_DowJ.csv")
NYT_TB = pd.read_csv("data/TextB_NYT_Score_DowJ.csv")
NYT_B = pd.read_csv("data/BERT_NYT_Score_DowJ.csv")

In [239]:
# 감성 분석 점수들을 여러 조합으로 만듦
senti_score = ['NBC_N_Score', 'NBC_TB_Score', 'NBC_B_Score', 'NYT_N_Score', 'NYT_TB_Score', 'NYT_B_Score']
senti_vari = []
for i in range(len(senti_score)+1):
    for vari in combinations(senti_score, i):
        senti_vari.append(list(vari))

## Catboost
- case 1
    - NBC : TextBlob 
    - NYT : NLTK, BERT
- 정확도 : 0.71

In [241]:
cat_preds = pd.DataFrame({"acc" : []})
i = 0
for vari in senti_vari[1:2]:
    Senti = [NBC_N, NBC_TB, NBC_B, NYT_N, NYT_TB, NYT_B]
    Senti_Score = pd.DataFrame(columns = ["Prediction_Date"])
    for df in Senti:
        Senti_Score = pd.merge(Senti_Score, df, how = "outer", on = "Prediction_Date")
    Senti_Score = pd.merge(Senti_Score, Vari_13, how = "outer", left_on = "Prediction_Date", right_on = "Dow_Date")
    Senti_Score = Senti_Score[Senti_Score.columns.difference(["Dow_Date", "Prediction_Date"])]

    #뉴스가 없는 날은 즉, score가 없는 날은 중립의 의미로 0으로 처리
    Senti_Score.NYT_N_Score[Senti_Score.NYT_N_Score.isnull()]=0
    Senti_Score.NYT_TB_Score[Senti_Score.NYT_TB_Score.isnull()]=0

    difference = 1
    #Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    #Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: x/abs(x), axis = 0)
    Senti_Score = Senti_Score.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    Senti_Score = Senti_Score.apply(lambda x: x/abs(x), axis = 0)

    Senti_Score = Senti_Score.dropna()
    Senti_Score = Senti_Score.reset_index(drop=True)

    dele = ["DowJ"] + [i for i in senti_score if i not in vari]
    x = Senti_Score[Senti_Score.columns.difference(dele)]
    y = Senti_Score[["DowJ"]]

    scaler_x = MinMaxScaler()
    
    if vari == []:
        pass
    else: x[vari] = pd.DataFrame(scaler_x.fit_transform(x[vari]))

    x.columns = Senti_Score.columns.difference(dele)
    y.columns = ["DowJ"]

    
    x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
    y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

    
    model = CatBoostClassifier()
    model.fit(x.train, y.train)
    preds_class = model.predict(x.test)
    cat_preds.loc[i] = sum((preds_class == np.array(list(itertools.chain(*y.test.values)))))/len(y.test)
    i += 1

/tmp/ipykernel_2782737/4165704232.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: x[vari] = pd.DataFrame(scaler_x.fit_transform(x[vari]))
/tmp/ipykernel_2782737/4165704232.py:38: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2782737/4165704232.py:39: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)


Learning rate set to 0.013003
0:	learn: 0.6887856	total: 2.22ms	remaining: 2.22s
1:	learn: 0.6846705	total: 3.5ms	remaining: 1.75s
2:	learn: 0.6806390	total: 5.26ms	remaining: 1.75s
3:	learn: 0.6763713	total: 7.04ms	remaining: 1.75s
4:	learn: 0.6712916	total: 9.61ms	remaining: 1.91s
5:	learn: 0.6667402	total: 12.2ms	remaining: 2.02s
6:	learn: 0.6616051	total: 15ms	remaining: 2.12s
7:	learn: 0.6576330	total: 17.3ms	remaining: 2.15s
8:	learn: 0.6535636	total: 19.2ms	remaining: 2.11s
9:	learn: 0.6489370	total: 21ms	remaining: 2.08s
10:	learn: 0.6448282	total: 22.9ms	remaining: 2.06s
11:	learn: 0.6413059	total: 24.8ms	remaining: 2.04s
12:	learn: 0.6375218	total: 26.6ms	remaining: 2.02s
13:	learn: 0.6339614	total: 28ms	remaining: 1.97s
14:	learn: 0.6308911	total: 28.9ms	remaining: 1.9s
15:	learn: 0.6270141	total: 30.2ms	remaining: 1.86s
16:	learn: 0.6234471	total: 31.6ms	remaining: 1.83s
17:	learn: 0.6199918	total: 33.1ms	remaining: 1.8s
18:	learn: 0.6169887	total: 34.4ms	remaining: 1.78s
1

In [230]:
sum((preds_class == np.array(list(itertools.chain(*y.test.values)))))/len(y.test)

0.7135135135135136

In [220]:
senti_vari[cat_preds.loc[cat_preds.acc == max(cat_preds.acc.to_list()),].index[0]]

['NBC_TB_Score', 'NYT_N_Score', 'NYT_B_Score']

## XGBoost
- case 1
    - NBC : NLTK, TextBlob, Bert
- case 2
    - NBC : TextBlob, BERT
    - NYT : TextBlob
- 정확도 : 0.76

In [ ]:
model=XGBClassifier()
param_grid={'booster' :['gbtree'],
                 'max_depth':[3,5,7,9,11],
                 'min_child_weight':[3,5,7],
                 'gamma':[0,2,4,6,8,10],
                 'nthread':[50],
                 'colsample_bytree':[0.3,0.5,0.7, 0.9],
                 'n_estimators':[50, 100, 150],
                 'objective':['binary:logistic'],
                 'random_state':[42]}
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))
x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)
cv=KFold(n_splits=6, shuffle = True, random_state=42)
gcv=GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='accuracy')
gcv.fit(x.train, y.train)

print('final params', gcv.best_params_)   # 최적의 파라미터 값 출력
print('best score', gcv.best_score_) 

/home/whfhrs3260/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<timed exec>:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [151]:

xgb_preds = pd.DataFrame({"acc" : []})
i = 0
for vari in senti_vari:
    Senti = [NBC_N, NBC_TB, NBC_B, NYT_N, NYT_TB, NYT_B]
    Senti_Score = pd.DataFrame(columns = ["Prediction_Date"])
    for df in Senti:
        Senti_Score = pd.merge(Senti_Score, df, how = "outer", on = "Prediction_Date")
    Senti_Score = pd.merge(Senti_Score, Vari_13, how = "outer", left_on = "Prediction_Date", right_on = "Dow_Date")
    Senti_Score = Senti_Score[Senti_Score.columns.difference(["Dow_Date", "Prediction_Date"])]

    #뉴스가 없는 날은 즉, score가 없는 날은 중립의 의미로 0으로 처리
    Senti_Score.NYT_N_Score[Senti_Score.NYT_N_Score.isnull()]=0
    Senti_Score.NYT_TB_Score[Senti_Score.NYT_TB_Score.isnull()]=0

    difference = 1
    #Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    #Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: x/abs(x), axis = 0)
    Senti_Score = Senti_Score.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    Senti_Score = Senti_Score.apply(lambda x: x/abs(x), axis = 0)

    Senti_Score = Senti_Score.dropna()
    Senti_Score = Senti_Score.reset_index(drop=True)

    dele = ["DowJ"] + [i for i in senti_score if i not in vari]
    x = Senti_Score[Senti_Score.columns.difference(dele)]
    y = Senti_Score[["DowJ"]]

    scaler_x = MinMaxScaler()
    
    if vari == []:
        pass
    else: x[vari] = pd.DataFrame(scaler_x.fit_transform(x[vari]))

    x.columns = Senti_Score.columns.difference(dele)
    y.columns = ["DowJ"]
    
    
    le = LabelEncoder()
    y = pd.DataFrame(le.fit_transform(y))

    x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
    y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

    model = XGBClassifier(colsample_bytree= 0.5, gamma= 8, max_depth= 5, min_child_weight= 5, n_estimators= 50, nthread= 50, random_state= 42)
    model.fit(x.train, y.train)
    preds_class = model.predict(x.test)
    xgb_preds.loc[i] = sum((preds_class == np.array(list(itertools.chain(*y.test.values)))))/len(y.test)
    i += 1

/home/whfhrs3260/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/tmp/ipykernel_2782737/474853995.py:41: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2782737/474853995.py:42: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2782737/474853995.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [152]:
max(xgb_preds.values)

array([0.75962815])

In [213]:
senti_vari[xgb_preds.loc[xgb_preds.acc == max(xgb_preds.acc.to_list()),].index[0]]

['NBC_N_Score', 'NBC_TB_Score', 'NBC_B_Score']

In [218]:
senti_vari[xgb_preds.loc[xgb_preds.acc == max(xgb_preds.acc.to_list()),].index[1]]

['NBC_TB_Score', 'NBC_B_Score', 'NYT_TB_Score']

## SVC
- case 1
    - NBC : TextBlob
    - NYT : NLTK, TextBlob, BERT
- case 2
    - NBC : TextBlob
    - NYT : TextBlob, BERT
- 정확도 : 0.74

In [ ]:
model = SVC()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
             'gamma': [0.001, 0.01, 0.1, 1, 10, 100] }

cv=KFold(n_splits=6, shuffle = True, random_state=42)
gcv = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy')

# train_test_split
gcv.fit(x.train, y.train)

print('final params', gcv.best_params_)   # 최적의 파라미터 값 출력
print('best score', gcv.best_score_)

In [263]:
svc_preds = pd.DataFrame({"acc" : []})
i = 0
for vari in senti_vari:
    Senti = [NBC_N, NBC_TB, NBC_B, NYT_N, NYT_TB, NYT_B]
    Senti_Score = pd.DataFrame(columns = ["Prediction_Date"])
    for df in Senti:
        Senti_Score = pd.merge(Senti_Score, df, how = "outer", on = "Prediction_Date")
    Senti_Score = pd.merge(Senti_Score, Vari_13, how = "outer", left_on = "Prediction_Date", right_on = "Dow_Date")
    Senti_Score = Senti_Score[Senti_Score.columns.difference(["Dow_Date", "Prediction_Date"])]

    #뉴스가 없는 날은 즉, score가 없는 날은 중립의 의미로 0으로 처리
    Senti_Score.NYT_N_Score[Senti_Score.NYT_N_Score.isnull()]=0
    Senti_Score.NYT_TB_Score[Senti_Score.NYT_TB_Score.isnull()]=0

    difference = 1
    Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: x/abs(x), axis = 0)
    #Senti_Score = Senti_Score.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    #Senti_Score = Senti_Score.apply(lambda x: x/abs(x), axis = 0)

    Senti_Score = Senti_Score.dropna()
    Senti_Score = Senti_Score.reset_index(drop=True)

    dele = ["DowJ"] + [i for i in senti_score if i not in vari]
    x = Senti_Score[Senti_Score.columns.difference(dele)]
    y = Senti_Score[["DowJ"]]

    scaler_x = MinMaxScaler()
    
    if vari == []:
        pass
    else: x[vari] = pd.DataFrame(scaler_x.fit_transform(x[vari]))

    x.columns = Senti_Score.columns.difference(dele)
    y.columns = ["DowJ"]

    
    x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
    y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

    
    model = SVC(kernel = "rbf", C = 0.1, gamma = 0.1)
    model.fit(x.train, y.train)
    preds_class = model.predict(x.test)
    svc_preds.loc[i] = sum((preds_class == np.array(list(itertools.chain(*y.test.values)))))/len(y.test)
    i += 1

/tmp/ipykernel_2782737/1874135939.py:38: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2782737/1874135939.py:39: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)
/home/whfhrs3260/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/tmp/ipykernel_2782737/1874135939.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [264]:
max(svc_preds.acc.to_list())

0.7423638778220452

In [209]:
senti_vari[svc_preds.loc[svc_preds.acc == max(svc_preds.acc.to_list()),].index[1]]

['NBC_TB_Score', 'NYT_N_Score', 'NYT_TB_Score', 'NYT_B_Score']

In [265]:
senti_vari[svc_preds.loc[svc_preds.acc == max(svc_preds.acc.to_list()),].index[0]]

['NBC_TB_Score', 'NYT_TB_Score', 'NYT_B_Score']

In [391]:
# AttributeError: 'SVC' object has no attribute '_validate_params'
#https://github.com/davidsandberg/facenet/issues/860

Requirement already up-to-date: scikit-learn==0.24.1 in /home/whfhrs3260/.local/lib/python3.8/site-packages (0.24.1)


## 나이브베이즈
- case 1
    - NBC : TextBlob
    - NYT : NTLK, TextBlob
- 정확도 : 0.74

In [157]:
NB_preds = pd.DataFrame({"acc" : []})
i = 0
for vari in senti_vari:
    Senti = [NBC_N, NBC_TB, NBC_B, NYT_N, NYT_TB, NYT_B]
    Senti_Score = pd.DataFrame(columns = ["Prediction_Date"])
    for df in Senti:
        Senti_Score = pd.merge(Senti_Score, df, how = "outer", on = "Prediction_Date")
    Senti_Score = pd.merge(Senti_Score, Vari_13, how = "outer", left_on = "Prediction_Date", right_on = "Dow_Date")
    Senti_Score = Senti_Score[Senti_Score.columns.difference(["Dow_Date", "Prediction_Date"])]

    #뉴스가 없는 날은 즉, score가 없는 날은 중립의 의미로 0으로 처리
    Senti_Score.NYT_N_Score[Senti_Score.NYT_N_Score.isnull()]=0
    Senti_Score.NYT_TB_Score[Senti_Score.NYT_TB_Score.isnull()]=0

    difference = 1
    Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: x/abs(x), axis = 0)
    #Senti_Score = Senti_Score.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    #Senti_Score = Senti_Score.apply(lambda x: x/abs(x), axis = 0)

    Senti_Score = Senti_Score.dropna()
    Senti_Score = Senti_Score.reset_index(drop=True)

    dele = ["DowJ"] + [i for i in senti_score if i not in vari]
    x = Senti_Score[Senti_Score.columns.difference(dele)]
    y = Senti_Score[["DowJ"]]

    scaler_x = MinMaxScaler()
    
    if vari == []:
        pass
    else: x[vari] = pd.DataFrame(scaler_x.fit_transform(x[vari]))

    x.columns = Senti_Score.columns.difference(dele)
    y.columns = ["DowJ"]

    
    x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
    y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

    
    model = GaussianNB()
    model.fit(x.train, y.train)
    preds_class = model.predict(x.test)
    NB_preds.loc[i] = sum((preds_class == np.array(list(itertools.chain(*y.test.values)))))/len(y.test)
    i += 1

/tmp/ipykernel_2782737/615891021.py:38: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2782737/615891021.py:39: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)
/home/whfhrs3260/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/tmp/ipykernel_2782737/615891021.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [158]:
max(NB_preds.acc.to_list())

0.7423638778220452

In [182]:
senti_vari[NB_preds.idxmax()[0]]

['NBC_TB_Score', 'NYT_N_Score', 'NYT_TB_Score']

## 랜덤포레스트
- case 1
    - NBC : NLTK, BERT
    - NYT : TextBlob, BERT
- 정확도 : 0.76

In [ ]:
# https://gils-lab.tistory.com/89
model = RandomForestClassifier() 
param_grid = {'max_depth': [5, 10, 15, 20],
             'max_samples': [0.2, 0.4, 0.7, 0.8, 0.9, 1.0],
             'max_features': [0.2, 0.4, 0.7, 0.8, 0.9, 1.0],
             "n_estimators" : [50, 100, 150, 200, 400]}

cv=KFold(n_splits=6, shuffle = True, random_state=42)
gcv = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy')

# train_test_split
gcv.fit(x.train, y.train)

print('final params', gcv.best_params_)   # 최적의 파라미터 값 출력
print('best score', gcv.best_score_)

In [159]:
RF_preds = pd.DataFrame({"acc" : []})
i = 0
for vari in senti_vari:
    Senti = [NBC_N, NBC_TB, NBC_B, NYT_N, NYT_TB, NYT_B]
    Senti_Score = pd.DataFrame(columns = ["Prediction_Date"])
    for df in Senti:
        Senti_Score = pd.merge(Senti_Score, df, how = "outer", on = "Prediction_Date")
    Senti_Score = pd.merge(Senti_Score, Vari_13, how = "outer", left_on = "Prediction_Date", right_on = "Dow_Date")
    Senti_Score = Senti_Score[Senti_Score.columns.difference(["Dow_Date", "Prediction_Date"])]

    #뉴스가 없는 날은 즉, score가 없는 날은 중립의 의미로 0으로 처리
    Senti_Score.NYT_N_Score[Senti_Score.NYT_N_Score.isnull()]=0
    Senti_Score.NYT_TB_Score[Senti_Score.NYT_TB_Score.isnull()]=0

    difference = 1
    Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: x/abs(x), axis = 0)
    #Senti_Score = Senti_Score.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    #Senti_Score = Senti_Score.apply(lambda x: x/abs(x), axis = 0)

    Senti_Score = Senti_Score.dropna()
    Senti_Score = Senti_Score.reset_index(drop=True)

    dele = ["DowJ"] + [i for i in senti_score if i not in vari]
    x = Senti_Score[Senti_Score.columns.difference(dele)]
    y = Senti_Score[["DowJ"]]

    scaler_x = MinMaxScaler()
    
    if vari == []:
        pass
    else: x[vari] = pd.DataFrame(scaler_x.fit_transform(x[vari]))

    x.columns = Senti_Score.columns.difference(dele)
    y.columns = ["DowJ"]

    
    x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
    y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

    
    model = RandomForestClassifier(max_depth = 5, max_features= 0.2, max_samples= 0.4, n_estimators= 400) 
    model.fit(x.train, y.train)
    preds_class = model.predict(x.test)
    RF_preds.loc[i] = sum((preds_class == np.array(list(itertools.chain(*y.test.values)))))/len(y.test)
    i += 1

/tmp/ipykernel_2782737/219392893.py:38: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2782737/219392893.py:39: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2782737/219392893.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x.train, y.train)
/tmp/ipykernel_2782737/219392893.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [172]:
max(RF_preds.acc.to_list())

0.7583001328021248

In [181]:
senti_vari[RF_preds.idxmax()[0]]

['NBC_N_Score', 'NBC_B_Score', 'NYT_TB_Score', 'NYT_B_Score']

## LSTM
- 정확도 : 0.73

In [145]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout

Senti = [NBC_N, NBC_TB, NBC_B, NYT_N, NYT_TB, NYT_B]
Senti_Score = pd.DataFrame(columns = ["Prediction_Date"])
for df in Senti:
    Senti_Score = pd.merge(Senti_Score, df, how = "outer", on = "Prediction_Date")
Senti_Score = pd.merge(Senti_Score, Vari_13, how = "outer", left_on = "Prediction_Date", right_on = "Dow_Date")
Senti_Score = Senti_Score[Senti_Score.columns.difference(["Dow_Date", "Prediction_Date"])]

#뉴스가 없는 날은 즉, score가 없는 날은 중립의 의미로 0으로 처리
Senti_Score.NYT_N_Score[Senti_Score.NYT_N_Score.isnull()]=0
Senti_Score.NYT_TB_Score[Senti_Score.NYT_TB_Score.isnull()]=0

difference = 1
Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
Senti_Score[Senti_Score.columns.difference(senti_score)] = Senti_Score[Senti_Score.columns.difference(senti_score)].apply(lambda x: x/abs(x), axis = 0)
#Senti_Score = Senti_Score.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
#Senti_Score = Senti_Score.apply(lambda x: x/abs(x), axis = 0)

Senti_Score = Senti_Score.dropna()
Senti_Score = Senti_Score.reset_index(drop=True)

dele = ["DowJ"] + [i for i in senti_score if i not in vari]
x = Senti_Score[Senti_Score.columns.difference(dele)]
y = Senti_Score[["DowJ"]]

scaler_x = MinMaxScaler()

if vari == []:
    pass
else: x[vari] = pd.DataFrame(scaler_x.fit_transform(x[vari]))

x.columns = Senti_Score.columns.difference(dele)
y.columns = ["DowJ"]

le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))
x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

y_train = np_utils.to_categorical(y.train)
y_test = np_utils.to_categorical(y.test)

x_train = x.train.values.reshape(x.train.shape[0],1, x.train.shape[1])
x_test = x.test.values.reshape(x.test.shape[0],1, x.test.shape[1])

model = Sequential()
model.add(LSTM(64, input_shape=(x_train.shape[1], x_train.shape[2]), activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(16))
model.add(Dense(2, activation='softmax'))  # 카테고리 수

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
18/18 [==============================] - 3s 7ms/step - loss: 0.6907 - accuracy: 0.5356
Epoch 2/100
18/18 [==============================] - 0s 9ms/step - loss: 0.6811 - accuracy: 0.5732
Epoch 3/100
18/18 [==============================] - 0s 9ms/step - loss: 0.6503 - accuracy: 0.6838
Epoch 4/100
18/18 [==============================] - 0s 9ms/step - loss: 0.5795 - accuracy: 0.7168
Epoch 5/100
18/18 [==============================] - 0s 9ms/step - loss: 0.5470 - accuracy: 0.7254
Epoch 6/100
18/18 [==============================] - 0s 9ms/step - loss: 0.5400 - accuracy: 0.7316
Epoch 7/100
18/18 [==============================] - 0s 9ms/step - loss: 0.5288 - accuracy: 0.7350
Epoch 8/100
18/18 [==============================] - 0s 9ms/step - loss: 0.5332 - accuracy: 0.7322
Epoch 9/100
18/18 [==============================] - 0s 9ms/step - loss: 0.5313 - accuracy: 0.7322
Epoch 10/100
18/18 [==============================] - 0s 9ms/step - loss: 0.5286 - accuracy: 0.7362
Epoch 11/

In [146]:
pred = model.predict(x_test)
pred = list(map(lambda x: 0 if x[0]>x[1] else 1, pred))
preds = pd.DataFrame({"acc" : []})
preds.loc[i] = sum((pred == np.array(list(itertools.chain(*y.test.values)))))/len(y.test)

24/24 [==============================] - 1s 3ms/step


In [148]:
preds

,acc
64,0.734396


In [164]:
pred = [cat_preds, xgb_preds, svc_preds, NB_preds, RF_preds]
preds = pd.DataFrame()
for i in pred:
    preds = pd.concat([preds, i], axis = 1)